In [ ]:
from pathlib import Path
import pandas as pd
import os
from dotenv import load_dotenv

In [10]:
project_folder = Path(r'C:\Users\timov\Desktop\Utrecht\Utrecht\MSBBSS\Thesis\Runs\Presentation')
location_search_terms = Path(r'Synergy\.synergy_dataset_source\synergy-dataset-1.0\Brouwer_2019\search_terms.txt')
n_runs = 4

## Import string

In [11]:
terms_path = Path(location_search_terms)
search_terms = terms_path.read_text(encoding="utf-8").strip()
print(search_terms)

Supplementary data: Search terms for each theory and all databases
1. Search cognitive: relapse/recurrence
1.1 Pubmed
 (("depressive disorder"[MeSH Terms] OR "depression"[MeSH Terms] OR depression[Title/Abstract] OR depressive[Title/Abstract] OR depressed[Title/Abstract] OR affective[Title/Abstract]) AND (recurrence[Title/Abstract] OR "recurrence"[MeSH Terms] OR recurrent[Title/Abstract] OR relapse[Title/Abstract] OR "recurrence"[MeSH Terms] OR remission[Title/Abstract] OR Predictor [Title/Abstract] OR prognostic [Title/Abstract] OR prescriptive [Title/Abstract] OR mediator [Title/Abstract] OR moderator [Title/Abstract] OR intervening [Title/Abstract])) AND ("cognitive theory" [Title/Abstract] OR "cognitive model"[ Title/Abstract] OR "cognitive therapy"[MeSH Terms] OR "cognitive therapy"[ Title/Abstract] OR "cognitive intervention" [ Title/Abstract] OR cognitive [Title/Abstract] OR hopelessness[Title/Abstract] OR helplessness[Title/Abstract] OR "dual processing"[ Title/Abstract] OR "in

## Define LLM

In [ ]:
import dspy

# 1) define back end
lm = dspy.LM("openai/gpt-4o-mini",  temperature=0 )
dspy.configure(lm=lm)

In [13]:
class MakeAbstract(dspy.Signature):
    """Generate one JSONL line with a synthetic title+abstract for a paper matched by the given search terms."""
    search_terms: str = dspy.InputField(desc="Boolean/keyword query used in the review")
    label_included: int = dspy.InputField(desc="1 if it would perfectly fit the review; 0 if it would be returned by the given search terms but not fit the review")
    extra_instructions: str = dspy.InputField(desc="")
    jsonl: str = dspy.OutputField(desc='One-line JSON object: {"doi":"None","title":"...","abstract":"...","label_included":"1/0","reasoning":"..."}')

make_abstract = dspy.ChainOfThought(MakeAbstract)

## Loop over to create several pairs of abstracts

In [5]:
for i in range(1,n_runs):

    #generate included abstract
    included = make_abstract(
        search_terms= search_terms,
        label_included=1,
        extra_instructions=""
    ).jsonl

     #generate excluded abstract
    excluded = make_abstract(
        search_terms= search_terms,
        label_included=0,
        extra_instructions=""
    ).jsonl
    
    # JSONL to CSV
    df_included = pd.read_json(included, lines=True)
    df_excluded = pd.read_json(excluded, lines=True)
    df = pd.concat([df_included, df_excluded])

    # create directory
    run_dir = project_folder / f"{location_search_terms.parent.name}_{i}" / "data"
    run_dir.mkdir(parents=True, exist_ok=True)
    csv_path = run_dir / f"prior_{location_search_terms.parent.name}.csv"

    #export csv
    df.to_csv(csv_path, index=False)


NameError: name 'n_runs' is not defined

In [2]:
df.abstract

NameError: name 'df' is not defined